# Advanced Regression Techniques 

While multiple regression is better than univariate regression because the former finds a model solution with **lower SSE** (higher r-squared). But, if there are **too many X variables (e.g., > 10) in a regression model**, the model becomes **too complex (or overfitting) to be useful** in practice due to **multicollinearity and difficulity of interpretation**. This problem is also known as **"curse of high dimensionality"**

Thus, **more advanced regression would be needed to deal with this issue**

> ## 1. Regularization 
> - refers to **the process of penalizing the model with too many redundant variables (or highly correlated variables)**
> - the goal is developing the regression model with **low SSE** and **simplicity (fewer X variables or predictors)** 
> - **two types of regression include regulariation in their objective function: Lasso and Ridge**
> - http://scikit-learn.org/stable/modules/linear_model.html

> ## 2. Feature selection
> - refers to **the process of selecting the most useful predictors**, helping analysts understand what predictors matter in predicting y value
> - the goal is developing the **simple** regression model with the **user specificed number of predictors**.
> - f_regression 

# Lasso regression (Regularization)

- (Least Absolute Shrinkage and Selection Operator) is one of the **regression models with regularization**
- Finds the model solution with **fewer X variables**

> #### How does Lasso work?#### 

> - Remember that the goal of regression is **Minimize SSE** and more predictors is likely to reduce **SSE**
> - Thus, Lasso includes **regularization** or **a mechanism of penalizing adding too many variables**. 

                  minimize (SSE  + alpha|coefficient|)

                  where alpha = parameter for penalizing adding more coefficients

> - This approach reinforces the Lasso regression to consider fewer predictors (simpler regression model) 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#regression packages
import sklearn.linear_model as lm
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
import statsmodels.formula.api as sm

#lasso regression
from sklearn import linear_model

#f_regression (feature selection)
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectKBest

# recursive feature selection (feature selection)
from sklearn.feature_selection import RFE

In [3]:
teams = pd.read_csv("data/baseball.csv")
teams.head()

,yearID,teamID,Rank,R,RA,G,W,H,BB,HBP,AB,SF,HR,2B,3B,salary,BA,OBP,SLG
0,2000,CHA,1,978,839,162,95,1615,591,53,5646,61,216,325,33,31133500,0.286043,0.355692,0.470067
1,2000,CLE,2,950,816,162,90,1639,685,51,5683,52,221,310,30,75880771,0.288404,0.367022,0.470174
2,2000,DET,3,823,827,162,79,1553,562,43,5644,49,177,307,41,58265167,0.275159,0.342648,0.438164
3,2000,KCA,4,879,930,162,77,1644,511,48,5709,70,150,281,27,23433000,0.287966,0.347586,0.425469
4,2000,MIN,5,748,880,162,69,1516,556,35,5615,51,116,325,49,16519500,0.269991,0.336743,0.407302


On Base Percentage (OBP, On Base Average, OBA) is a measure of how often a batter reaches base. 

The full formula is OBP = (Hits + Walks + Hit by Pitch) / (At Bats + Walks + Hit by Pitch + Sacrifice Flies). Batters are not credited with reaching base on an error or fielder's choice, and they are not charged with an opportunity if they make a sacrifice bunt.

All Time Leaders
Ted Williams	.482	(career)
Barry Bonds	    .609	(2004 season)

http://www.baseball-reference.com/bullpen/On_base_percentage

In [4]:
teams = teams.drop(['yearID', 'teamID', 'Rank'], axis=1)
teams.head(2)

,R,RA,G,W,H,BB,HBP,AB,SF,HR,2B,3B,salary,BA,OBP,SLG
0,978,839,162,95,1615,591,53,5646,61,216,325,33,31133500,0.286043,0.355692,0.470067
1,950,816,162,90,1639,685,51,5683,52,221,310,30,75880771,0.288404,0.367022,0.470174


In [5]:
#assigning columns to X and Y variables
y = teams['R'] 
X = teams.drop(['R'], axis =1)

In [6]:
#Fit the model
model1 = linear_model.Lasso(alpha=1)         #higher alpha (penality parameter), fewer predictors
model1.fit(X, y)
model1_y = model1.predict(X)

In [7]:
print 'Coefficients: ', model1.coef_
print "y-intercept ", model1.intercept_

Coefficients:  [  2.12210981e-01   0.00000000e+00   2.11287738e+00   4.70584673e-01
   2.42458543e-01   2.96796937e-01  -5.42432118e-02   5.57796970e-01
   7.16499037e-01   1.64546847e-01   6.17523323e-01  -2.56238003e-08
  -0.00000000e+00  -0.00000000e+00   0.00000000e+00]
y-intercept  -321.284856921


In [8]:
coef = ["%.3f" % i for i in model1.coef_]
xcolumns = [ i for i in X.columns ]
zip(xcolumns, coef)

[('RA', '0.212'),
 ('G', '0.000'),
 ('W', '2.113'),
 ('H', '0.471'),
 ('BB', '0.242'),
 ('HBP', '0.297'),
 ('AB', '-0.054'),
 ('SF', '0.558'),
 ('HR', '0.716'),
 ('2B', '0.165'),
 ('3B', '0.618'),
 ('salary', '-0.000'),
 ('BA', '-0.000'),
 ('OBP', '-0.000'),
 ('SLG', '0.000')]

The regression model has become a lot simpler than the full model with all X variables. Several X variables were removed from the model, including **G, AB, salary, BA, OBP, and SLG**. These removed variables have their **coefficients close to 0**.

R = 0.212RA + 2.113W + 0.471H + 0.242BB + ...

In [9]:
print "mean square error: ", mean_squared_error(y, model1_y)
print "variance or r-squared: ", explained_variance_score(y, model1_y)

mean square error:  399.392501942
variance or r-squared:  0.942986315484


# f_Regression (Feature Selection)

- Quick linear model for testing the effect of a single predictor, sequentially for many predictors.
- http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression

In [10]:
#selec only 2 X variables
X_new = SelectKBest(f_regression, k=2).fit_transform(X, y)
X_new

array([[ 0.35569202,  0.4700673 ],
       [ 0.3670221 ,  0.4701742 ],
       [ 0.34264846,  0.43816442],
       [ 0.34758599,  0.42546856],
       [ 0.33674285,  0.40730187],
       [ 0.35414681,  0.449964  ],
       [ 0.34054652,  0.42344583],
       [ 0.34111482,  0.46926193],
       [ 0.34057971,  0.43503334],
       [ 0.32851105,  0.39909174],
       [ 0.35950671,  0.45773381],
       [ 0.36107193,  0.44151355],
       [ 0.35235536,  0.47245913],
       [ 0.35154394,  0.44617564],
       [ 0.35612083,  0.4554582 ],
       [ 0.34325522,  0.44702751],
       [ 0.32528206,  0.40337947],
       [ 0.36057617,  0.47666068],
       [ 0.3385103 ,  0.423888  ],
       [ 0.33481294,  0.41115295],
       [ 0.34647705,  0.42867553],
       [ 0.34601247,  0.43036821],
       [ 0.33133117,  0.40896714],
       [ 0.32597959,  0.43161698],
       [ 0.32903434,  0.39956451],
       [ 0.36170213,  0.4720058 ],
       [ 0.34053794,  0.43094326],
       [ 0.33338728,  0.42934684],
       [ 0.36171213,

f_regression determines that **OBP** and **SLG** are two most important predictors

In [ ]:
model2 = lm.LinearRegression()
model2.fit(X_new, y)
model2_y = model2.predict(X_new)

print "mean square error: ", mean_squared_error(y, model2_y)
print "variance or r-squared: ", explained_variance_score(y, model2_y)

In [ ]:
# use f_regression with k = 3 and develop a new regression model


model3





# Recursive Feature Selection (RFE): Another Feature Selection Method

In [11]:
lr = lm.LinearRegression()
rfe = RFE(lr, n_features_to_select=2)
rfe_y = rfe.fit(X,y)

print "Features sorted by their rank:"
print sorted(zip(map(lambda x: x, rfe.ranking_), X.columns))

Features sorted by their rank:
[(1, 'OBP'), (1, 'SLG'), (2, 'BA'), (3, 'HR'), (4, '3B'), (5, '2B'), (6, 'H'), (7, 'AB'), (8, 'SF'), (9, 'HBP'), (10, 'BB'), (11, 'W'), (12, 'RA'), (13, 'G'), (14, 'salary')]


In [12]:
# or you can do something like this: zip first and sort the data

print sorted(zip(rfe.ranking_, X.columns))

[(1, 'OBP'), (1, 'SLG'), (2, 'BA'), (3, 'HR'), (4, '3B'), (5, '2B'), (6, 'H'), (7, 'AB'), (8, 'SF'), (9, 'HBP'), (10, 'BB'), (11, 'W'), (12, 'RA'), (13, 'G'), (14, 'salary')]


# Appendix: with fewer predictors

In [ ]:
#First Model
runs_reg_model1 = sm.ols("R~OBP+SLG+BA",teams)
runs_reg1 = runs_reg_model1.fit()
#Second Model
runs_reg_model2 = sm.ols("R~OBP+SLG",teams)
runs_reg2 = runs_reg_model2.fit()
#Third Model
runs_reg_model3 = sm.ols("R~BA",teams)
runs_reg3 = runs_reg_model3.fit()

- The first one will have as features OBP, SLG and BA. 
- The second model will have as features OBP and SLG. 
- The last one will have as feature BA only.

In [ ]:
print runs_reg1.summary()
print runs_reg2.summary()
print runs_reg3.summary()

- The first model has an Adjusted R-squared of 0.918, with 95% confidence interval of BA between -283 and 468. This is counterintuitive, since we expect the BA value to be positive. This is due to a **multicollinearity** between the variables.

- The second model has an Adjusted R-squared of 0.919, and the last model an Adjusted R-squared of 0.500.

- Based on this analysis, we could confirm that the second model using **OBP** and **SLG** is the best model for predicting Run Scored.

# References

- http://adilmoujahid.com/posts/2014/07/baseball-analytics/ (reproduced from this page)
- https://www.analyticsvidhya.com/blog/2016/01/complete-tutorial-ridge-lasso-regression-python/#four
- http://www.python-course.eu/lambda.php (excellent resource for lamda and map)